## Optical Flow using OpenCV

In [1]:
import cv2
import numpy as np

### 1. lucas Kanade 
- maxCorners: This parameter specifies the maximum number of corners to detect. It determines the total number of feature points that the algorithm will try to find in the image. Increasing this value allows more corners to be detected, potentially capturing more details, but it can also lead to increased computational complexity.
</br>
</br>
- qualityLevel: This parameter defines the minimum accepted quality level for a corner to be considered. It represents a value between 0 and 1, indicating the minimum "strength" or "cornerness" of a corner. Corners with a higher "strength" value (closer to 1) are more likely to be selected as feature points. Adjusting this parameter affects the sensitivity of the corner detection algorithm. A higher value results in fewer selected corners but potentially higher quality.
</br>
</br>
- minDistance: This parameter sets the minimum Euclidean distance between detected corners. It ensures that the selected corners are spread out across the image and are not too close to each other. A larger minDistance value enforces a greater spatial distribution of feature points, preventing them from being too densely packed.
</br>
</br>
- blockSize: This parameter defines the size of the neighborhood considered for corner detection. It specifies the size of the Sobel operator aperture used to calculate the gradient at each pixel. A larger blockSize captures corners with larger-scale variations in intensity, while a smaller value focuses on smaller-scale variations. Adjusting this parameter can impact the size and scale of the detected corners.
</br>
</br>
- winSize: This parameter defines the size of the search window for each pyramid level. The window size determines the spatial extent within which the algorithm searches for corresponding points in the next frame. A larger window size can capture larger motion but may result in decreased accuracy. Conversely, a smaller window size can provide more precise estimates but may struggle with larger displacements or occlusions.
</br>
</br>
- maxLevel: This parameter specifies the maximum pyramid level for the iterative algorithm. The Lucas-Kanade algorithm operates on image pyramids, where each level represents a different scale of the image. By limiting the maximum pyramid level, you control the level of detail considered during the optical flow estimation. Higher pyramid levels capture finer details but increase computational complexity.
</br>
</br>
- criteria: This parameter defines the termination criteria for the iterative algorithm. It is specified as a tuple (type, maxCount, epsilon), where: 
 - type specifies the type of termination criteria. Here, we use a combination of cv2.TERM_CRITERIA_EPS and cv2.TERM_CRITERIA_COUNT, which indicate that the algorithm terminates based on a maximum number of iterations (maxCount) or a specified change in the estimated points (epsilon).
 - maxCount determines the maximum number of iterations before the algorithm terminates.
 - epsilon sets the required accuracy for the estimated points. A smaller value indicates higher precision.

In [45]:
cap = cv2.VideoCapture('Videos\Video2.mp4')

# params for ShiTomasi Corner Detection
feature_params = dict( maxCorners = 2500,
                     qualityLevel = 0.7,
                     minDistance = 5,
                     blockSize = 2)

# Parameters for lucas kanade optical flow
lk_params = dict( winSize = (40,40),
                maxLevel = 5,
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 20, 0.03))

# create random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while True:
    ret, frame = cap.read()
    
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate Optical Flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray,
                                              gray, p0,
                                              None, **lk_params)

        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]

        # Draw the Tracks
        # Iterating through all points from detected points
        for i,(new,old) in enumerate(zip(good_new,good_old)):
            a,b = new.ravel()
            c,d = old.ravel()

            mask = cv2.line(mask, (int(a),int(b)), (int(c),int(d)), color[i].tolist(), 2)
            frame = cv2.circle(frame, (int(a),int(b)), 5, color[i].tolist(), -1)

        img = cv2.add(frame,mask)
        cv2.imshow('Optical Flow',img)
        
        old_gray = gray.copy()
        p0 = good_new.reshape(-1,1,2)
    
    if cv2.waitKey(15) & 0xFF == 27:
        break
        
    
        
cap.release()
cv2.destroyAllWindows()

In [3]:
cv2.destroyAllWindows()

### 2. Gunner Farneback's Algorithm

- prvs: The previous frame (gray-scale) used as input for optical flow calculation.
</br>
</br>
- new: The current frame (gray-scale) used as input for optical flow calculation.
</br>
</br>
- None: Placeholder for the output flow image (not used in this case).
</br>
</br>
- 0.5: The pyramid scale factor. It controls the image scale in the pyramid. A smaller value leads to more levels in the pyramid and finer details captured.
</br>
</br>
- 3: The number of pyramid levels. It represents the number of times the image is downsampled in the pyramid. More levels allow capturing larger motion.
</br>
</br>
- 15: The window size. It specifies the size of the neighborhood used for calculating the optical flow. A larger window size captures more global motion but may lead to less accurate results for small motions.
</br>
</br>
- 3: The number of iterations of the algorithm at each pyramid level. More iterations can improve accuracy but also increase computational cost.
</br>
</br>
- 5: The size of the pixel neighborhood used to find polynomial expansion for each pixel. A larger value captures more local motion but may introduce more noise.
</br>
</br>
- 1.2: The standard deviation of the Gaussian used for smoothing derivatives used as a basis for the polynomial expansion. A larger value increases the influence of distant pixels and can capture larger motions.
</br>
</br>
- 0: Additional flags. Setting it to 0 means no additional options are enabled.

In [7]:
cap = cv2.VideoCapture('Videos\Video2.mp4')

# create random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
# Assigning 255 to Saturation Value in hsv Image
hsv[...,1] =  255

while True:
    ret, frame2 = cap.read()
    
    if ret:
        new = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Calculate Optical Flow
        # Parameters (prev, next, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags)
        flow = cv2.calcOpticalFlowFarneback(prvs, new, None, 
                                            0.5, 3, 15, 3, 5, 1.2, 0)

        mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
        hsv[...,0] = ang*180 / np.pi/2
        hsv[...,2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        img = cv2.add(frame2,rgb)
        
        cv2.imshow('Optical Flow',img)
        
        prvs = new
    
    if cv2.waitKey(25) & 0xFF == 27:
        break   
    
        
cap.release()
cv2.destroyAllWindows()

In [5]:
cv2.destroyAllWindows()

### Keypoint :
-  If you need dense flow estimation and want to capture detailed motion information, the Farneback method can be a good choice. If you are interested in sparse flow estimation and precise feature tracking, the Lucas-Kanade method with feature point tracking may be more suitable.